In [2]:
from numpy.linalg import eig
import numpy as np
import cv2
import os
import sys
import pickle

In [3]:
configuration_file = 'config/part_1.cfg'

**Main: process_video.py**

In [4]:
from src.extract_features import *
from src.parsing import *
from src.display_video import *

config_data = parse_configuration_file(configuration_file)     #sys.argv[1]) #Parse the configuration file
match_img1 , match_map = parse_points(config_data) #Parse the points from the configuration file
video_path = config_data[0].split(' ')[1].strip() #Get the video path
sift_points, img1, img2 = extract_features(video_path)


image matches:  [('225', '131'), ('580', '120'), ('626', '305'), ('133', '303')]
map matches:  [('225', '131'), ('580', '120'), ('626', '305'), ('133', '303')] 

video/trymefirst_lisbon.mp4
Total frames of the video:  1901
(Nº features, Nº descriptors per feature):  (5000, 128)
Nº of frames extracted:  20


Output of process_video.py:

In [ ]:
from src.outputs import *
create_output_keypoints(sift_points, 'outputs/keypoints.ext')


In [37]:
type_homography = 'all'

**Main: compute_transform.py**

In [ ]:
from src.matching_features import *
from src.homography import *
from src.ransac import *
from src.view_homography import *
from main import *

H_frame1_to_map =compute_homography(match_img1, match_map)    
match = matching_features_SCIKITLEARN(sift_points)
H_sequential = create_sequential_homographies(match, sift_points)
if type_homography =='map':
    H_output = homography_to_map(H_sequential, H_frame1_to_map)
elif type_homography =='all':
    H_output = all_homographies(H_sequential)



Output of compute_transform.py:

In [ ]:
from src.outputs import *
create_output(H_output, 'outputs/transformations.ext')        


**Display results:**

In [ ]:
def show_pixel_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at (x={x}, y={y})")

def display_frame(name,frame):
    cv2.namedWindow(name)
    cv2.setMouseCallback(name, show_pixel_coordinates)
    cv2.imshow(name, frame)

def display(frame1, frame2,homography_de2_para1 ):
    H= homography_de2_para1.reshape((3,3))
    
    height, width = frame2.shape[:2]
    # Apply homography to frame 2
    warped_frame2 = cv2.warpPerspective(frame2, H, (width, height))

    while True:
        display_frame('Frame 1', frame1)

        display_frame('Frame 2 with homography applied to frame 1 coordinates', warped_frame2)

        display_frame("Frame 2", frame2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

def extract__frames(video_path):
    """Extracts the features from the video and stores them in a list"""
    print(video_path)
    capture = cv2.VideoCapture(os.path.abspath(video_path))
    k = 0
    frames=[]
    count_frames(video_path)
    while k <= 1900:
        capture.set(cv2.CAP_PROP_POS_FRAMES, k)
        success, frame = capture.read() #read the video
        if success:
            frames.append(frame)
        k += 100
    return frames

In [ ]:
frames= extract__frames('video/trymefirst_lisbon.mp4')

In [ ]:
#to test homography
img1=frames[0]
for i in range(1,len(frames)-1):
    img2= frames[i]
    display(img1, img2, H_output[2:,i] )